**Импорт библиотек**

In [1]:
import os
import shutil
import json
from PreapreDataLib import create_folder_if_not_exists

import numpy as np
from pycocotools.coco import COCO
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.models import MaskRCNN
import mmdet.apis as mmapi
# from mmdet.apis import train_detector, init_detector, inference_detector, show_result_pyplot
import torch

D:\University\miniconda3\envs\RoadDamageDetection\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [8]:
DATASET_DIR = './DatasetCocoFormat1200/'
TRAIN_DIR = DATASET_DIR + 'train/'
TEST_DIR  = DATASET_DIR + 'test/'
VAL_DIR = DATASET_DIR + 'val/'
ANN_DIR = DATASET_DIR + 'annotations/'
TRAIN_ANN_PTH = ANN_DIR + "train.json"
VAL_ANN_PTH = ANN_DIR + "val.json"
TEST_ANN_PTH = ANN_DIR + "test.json"

**Загрузка конфига Mask R-CNN ResNet101 FPN 1x**

In [9]:
CONFIGS_DIR = "./configs/"
create_folder_if_not_exists("./configs/")
MRCNN_BASE_CONFIG_PTH = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco.py"
MRCNN_USER_CONFIG_PTH = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.py"

In [97]:
if not os.path.exists(MRCNN_BASE_CONFIG_PTH):
    !mim download mmdet --config mask_rcnn_r50_fpn_1x_coco --dest "./configs"

**Изменение конфига под задачу и датасет**

In [98]:
cfg = Config.fromfile(MRCNN_BASE_CONFIG_PTH)

In [99]:
cfg.model.roi_head.bbox_head.num_classes = 2
cfg.model.roi_head.mask_head.num_classes = 2

cfg.data_root = DATASET_DIR

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1

cfg.data.train.img_prefix = TRAIN_DIR
cfg.data.val.img_prefix = VAL_DIR
cfg.data.test.img_prefix = TEST_DIR

cfg.data.train.ann_file = TRAIN_ANN_PTH
cfg.data.val.ann_file = VAL_ANN_PTH
cfg.data.test.ann_file = TEST_ANN_PTH

# (1280, 640)
cfg.test_pipeline[1].img_scale = (640, 640)
cfg.train_pipeline[2].img_scale = (640, 640)

cfg.data.train.pipeline[2].img_scale = (640, 640)
cfg.data.val.pipeline[1].img_scale = (640, 640)
cfg.data.test.pipeline[1].img_scale = (640, 640)
# cfg.data.test.pipeline[1].img_scale = (512, 320)

# изменение гиперпараметров
cfg.model.rpn_head.anchor_generator.scales=[2,4,8,16,32]
cfg.model.test_cfg.rcnn.max_per_img = 200
cfg.model.test_cfg.rcnn.nms.iou_threshold = 0.3
# cfg.model.train_cfg.rcnn.mask_size = 56

cfg.optimizer.lr = 0.01
cfg.optimizer.weight_decay = 0.00001
cfg.runner.max_epochs = 300

cfg.lr_config.warmup_by_epoch = True
cfg.lr_config.warmup_iters = 32
cfg.lr_config.warmup_ratio = 0.0003125


cfg_classes = ("crack", "pothole")
cfg.data.train.classes = cfg_classes
cfg.data.val.classes = cfg_classes
cfg.data.test.classes = cfg_classes

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = "cuda"
cfg.load_from = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth"

cfg.work_dir = CONFIGS_DIR

In [100]:
if os.path.exists(MRCNN_USER_CONFIG_PTH):
    os.remove(MRCNN_USER_CONFIG_PTH)
cfg.dump(MRCNN_USER_CONFIG_PTH)

In [10]:
user_config = Config.fromfile(MRCNN_USER_CONFIG_PTH)
print(user_config.pretty_text)

model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[2, 4, 8, 16, 32],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, lo

**Создание модели**

In [102]:
model = build_detector(user_config.model)

In [11]:
datasets = [build_dataset(user_config.data.train)]

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!


**Обучение модели**

In [25]:
# model.with_cp = True
# model.fp16_enabled = True
mmapi.train_detector(model, datasets[0], cfg, distributed=False, validate=True)
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [31]:
torch.cuda.empty_cache()

**Сохранение результатов обучения**

In [12]:
MRCNN_RESNET50_FPN_RDD_PTH = CONFIGS_DIR + "mask_rcnn_r50_fpn_1x_coco_for_road_damages_5_epoch_25.pth"

In [7]:
if not os.path.exists(MRCNN_RESNET50_FPN_RDD_PTH):
    shutil.copy(CONFIGS_DIR + "epoch_25.pth", MRCNN_RESNET50_FPN_RDD_PTH)

In [5]:
# %matplotlib inline
from mmdetection.tools.analysis_tools.analyze_logs import plot_curve
# class PlotInput:
#     def __init__(self, backend=None, style=None, legend=None, json_logs=None, keys=None, eval_interval=None, title=None, out=None):
#         self.backend = backend
#         self.style = style
#         self.legend = legend
#         self.json_logs = json_logs
#         self.keys = keys
#         self.eval_interval = eval_interval
#         self.title = title
#         self.out = out

    # def backend(self):
    #     return self.backend
    #
    # def style(self):
    #     return self.style()


# input = PlotInput(keys="segm_mAP_50", legend="segm_mAP_50")
# a = input.keys

# plot_curve("configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json",
#            dict(backend: None, style: None, legend: segm_mAP_50, json_logs: None, keys: "segm_mAP_50", eval_interval: None, title: None, out: None))
# plot_curve("configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json", PlotInput)
!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json --keys segm_mAP_50  --legend segm_mAP_50

!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_3.log.json configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json --keys loss_mask --legend loss_mask_20_epochs loss_mask_268_epochs

!python mmdetection/tools/analysis_tools/analyze_logs.py plot_curve configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_3.log.json configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json --keys segm_mAP_50 segm_mAP_75 segm_mAP_s --legend segm_mAP_50_20_epochs segm_mAP_75_20_epochs segm_mAP_s_20_epochs segm_mAP_50_268_epochs segm_mAP_75_268_epochs segm_mAP_s_268_epochs


plot curve of configs/mask_rcnn_r50_fpn_1x_coco_for_road_damages_5.log.json, metric is segm_mAP_50
Figure(640x480)
